In [ ]:
from flask import Flask, render_template, Response, request

import cv2
import numpy as np
import time
from ultralytics import YOLO
from playsound import playsound
from threading import Thread

app = Flask(__name__)



model = YOLO("yolo11n.pt")  

# Constants
focal_length = 150  
known_width = 10.0 
distance_threshold = 0.10 

classNames = []
classFile = "dataset/coco.txt"
with open(classFile,"rt") as f:
    classNames = f.read().rstrip("\n").split("\n")


alarm_on = False
alarm_sound = "data/alarm.mp3"

def start_alarm(sound):
    """Play the alarm sound"""
    playsound('alarm.mp3')

def get_distance(known_width, focal_length, pixel_width):
    """Calculate object distance from the camera."""
    return (known_width * focal_length) / pixel_width

def detect_objects(img):
    """Perform object detection and calculate distance."""
    results = model(img)
    object_info = []

    for r in results:
        boxes = r.boxes  # Get bounding boxes

        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            pixel_width = x2 - x1  # Object width in pixels
            class_id = int(box.cls[0])  # Object class ID
            confidence = float(box.conf[0])  # Confidence score

            # Calculate distance
            distance = get_distance(known_width, focal_length, pixel_width) * 0.0254  # Convert to meters

            # Display bounding box and information
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"Object: {classNames[class_id]}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(img, f"Conf: {confidence:.2f}", (x1, y1 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(img, f"Dist: {distance:.2f} m", (x1, y1 + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Trigger an alarm if the object is too close
            if distance < distance_threshold:
                cv2.putText(img, "STOP! Object too close", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                #if not alarm_on:
                alarm_on = True
                print('alarm')
                t = Thread(target=start_alarm, args=(alarm_sound,))
                t.daemon = True
                t.start()

            object_info.append((class_id, confidence, distance))
            alarm_on = False

    return img, object_info

@app.route('/')
def index():
    return render_template('index.html')


def gen_frames():
    # Start video capture
    cap = cv2.VideoCapture(0)

    while True:
        success, frame = cap.read()
        if not success:
            break

        # Process frame
        result_frame, detected_objects = detect_objects(frame)
        
        #for r in result_frame:
        #    frame = r.plot()  # Draw detected objects on the frame

        # Convert frame to JPEG to send to frontend
        ret, buffer = cv2.imencode('.jpg', result_frame)
        if not ret:
            continue
        
        frame = buffer.tobytes()

        # Yield the frame for the video stream
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

        # Show the processed frame
        #cv2.imshow("YOLOv11 Distance-Based Detection", result_frame)

        # Exit on 'q' key
        #if cv2.waitKey(1) & 0xFF == ord('q'):
         #   break

    #cap.release()
    #cv2.destroyAllWindows()
    
@app.route('/live_camera')
def live_camera():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')


    

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Mar/2025 16:07:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2025 16:07:41] "GET /static/images/image.jpg HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2025 16:07:41] "GET /favicon.ico HTTP/1.1" 404 -



0: 480x640 1 vase, 438.3ms
Speed: 27.7ms preprocess, 438.3ms inference, 30.9ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [28/Mar/2025 16:10:34] "GET /live_camera HTTP/1.1" 200 -



0: 480x640 1 person, 302.8ms
Speed: 6.2ms preprocess, 302.8ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 368.7ms
Speed: 22.8ms preprocess, 368.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 261.5ms
Speed: 5.8ms preprocess, 261.5ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 2 persons, 382.2ms
Speed: 4.3ms preprocess, 382.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 232.4ms
Speed: 3.2ms preprocess, 232.4ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 2 persons, 203.3ms
Speed: 13.5ms preprocess, 203.3ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 184.6ms
Speed: 6.1ms preprocess, 184.6ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 2 persons, 209.5ms
Speed: 4.6ms preprocess, 20

...

127.0.0.1 - - [28/Mar/2025 16:30:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Mar/2025 16:30:29] "GET /static/images/image.jpg HTTP/1.1" 304 -



0: 480x640 (no detections), 758.6ms
Speed: 115.6ms preprocess, 758.6ms inference, 15.5ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [28/Mar/2025 16:36:09] "GET /live_camera HTTP/1.1" 200 -



0: 480x640 1 person, 491.5ms
Speed: 5.4ms preprocess, 491.5ms inference, 52.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 305.8ms
Speed: 4.7ms preprocess, 305.8ms inference, 13.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 260.4ms
Speed: 7.2ms preprocess, 260.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 251.1ms
Speed: 4.2ms preprocess, 251.1ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 234.3ms
Speed: 3.7ms preprocess, 234.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 233.4ms
Speed: 5.4ms preprocess, 233.4ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 donut, 224.7ms
Speed: 5.9ms preprocess, 224.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 228.9ms
Speed: 3.8ms preprocess, 228.9ms inference, 2.3ms postproce


0: 480x640 (no detections), 218.8ms
Speed: 4.7ms preprocess, 218.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 keyboard, 215.8ms
Speed: 4.2ms preprocess, 215.8ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 keyboard, 223.6ms
Speed: 4.7ms preprocess, 223.6ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 228.6ms
Speed: 7.4ms preprocess, 228.6ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 298.4ms
Speed: 7.0ms preprocess, 298.4ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 244.3ms
Speed: 5.1ms preprocess, 244.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 231.1ms
Speed: 5.3ms preprocess, 231.1ms inference, 3.8ms postprocess per image at shape (1, 3, 480, 640)
alarm

0: 480x640 1 person, 283.5ms
Speed: 7.5ms preprocess, 283.5ms infere